In [1]:
from model import vae,decoder
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Input
from keras.models import Model
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
import cv2
import numpy as np
import mdn
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.
/Users/brandonquach/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 208, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 120, 208, 128 1280        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 60, 104, 128) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 60, 104, 64)  73792       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [4]:
def data_generator(batch_size,limit):

	batch = []
	counter = 1
	while 1:
		for i in range(1,limit+1):
			if counter >= limit:
				counter = 1
			img = cv2.imread("imgs/{}.jpg".format(counter),cv2.IMREAD_GRAYSCALE)
			img = img.reshape(120,208,1)
			batch.append(img)
			if len(batch) == batch_size:
				batch_np = np.array(batch) / 255
				batch = []
				return batch_np
			counter += 1

In [5]:
import os
X = data_generator(batch_size = 47950, limit = len(os.listdir('imgs')))

In [6]:
data = X

# Gumbel Softmax VAE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Lambda
from keras.models import Model, Sequential
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.activations import softmax
from keras.objectives import binary_crossentropy as bce

batch_size = 100
data_dim = 24960
M = 80
N = 240

nb_epoch = 100
epsilon_std = 0.01

anneal_rate = 0.0003
min_temperature = 0.5

tau = K.variable(5.0, name="temperature")
x = Input(shape = (data_dim,))
h = Dense(256, activation='relu')(Dense(512, activation='relu')(x))
logits_y = Dense(M*N)(h)

def sampling(logits_y):
    U = K.random_uniform(K.shape(logits_y), 0, 1)
    y = logits_y - K.log(-K.log(U + 1e-20) + 1e-20) # logits + gumbel noise
    y = softmax(K.reshape(y, (-1, N, M)) / tau)
    y = K.reshape(y, (-1, N*M))
    return y

z = Lambda(sampling, output_shape=(M*N,))(logits_y)
generator = Sequential()
generator.add(Dense(256, activation='relu', input_shape=(N*M, )))
generator.add(Dense(512, activation='relu'))
generator.add(Dense(data_dim, activation='sigmoid'))
x_hat = generator(z)

# x_hat = Dense(data_dim, activation='softmax')(Dense(512, activation='relu')(Dense(256, activation='relu')(z)))
def gumbel_loss(x, x_hat):
    q_y = K.reshape(logits_y, (-1, N, M))
    q_y = softmax(q_y)
    log_q_y = K.log(q_y + 1e-20)
    kl_tmp = q_y * (log_q_y - K.log(1.0/M))
    KL = K.sum(kl_tmp, axis=(1, 2))
    elbo = data_dim * bce(x, x_hat) - KL 
    return elbo

model = Model(x, x_hat)
model.compile(optimizer='adam', loss=gumbel_loss)

X = data[0:len(data)-1]
X = X.reshape((len(X), np.prod(X.shape[1:])))
y = data[1:len(data)]
y = y.reshape((len(y), np.prod(y.shape[1:])))

y = y[:47900]
y_train = y[:38300]
y_test = y[38300:]
X = X[:47900]
X_train = X[:38300]
X_test = X[38300:]

for e in range(nb_epoch):
    model.fit(X_train, y_train,
        shuffle=False,
        nb_epoch=1,
        batch_size=batch_size,
        validation_data = (X_test, y_test))
    K.set_value(tau, np.max([K.get_value(tau) * np.exp(- anneal_rate * e), min_temperature]))

/Users/brandonquach/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:72: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 111s 15ms/step - loss: 5900.9438 - val_loss: 4198.8210
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 111s 15ms/step - loss: 4909.9179 - val_loss: 4176.0520
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 99s 13ms/step - loss: 4880.8507 - val_loss: 4172.3943
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 113s 15ms/step - loss: 4859.0903 - val_loss: 4140.7443
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 117s 16ms/step - loss: 4847.2770 - val_loss: 4147.5519
Train on 7500 samples, validate on 2500 samples
Epoch 1/1
7500/7500 [==============================] - 114s 15ms/step - loss: 4857.0861 - val_loss: 4155.5361
Train on 7500 samples, validate on 2500 samples
Epoch

# Generate Video

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter("out.mp4", fourcc, 30.0, (208, 120))
lv_in = X[0]

for i in range(500):
    temp = model.predict(lv_in.reshape(1, 24960))
    print(temp)
    img = temp
    img = np.array(img).reshape(120,208,1)
    img = img * 255
    img = np.array(img).astype("uint8")
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    lv_in = temp
    video.write(img)
video.release()


[[0.9993643  0.9992611  0.9993783  ... 0.99631715 0.99758744 0.99768126]]
[[0.9993648 0.9992617 0.9993789 ... 0.9963194 0.9975891 0.9976828]]
[[0.9993656  0.99926263 0.99937975 ... 0.996323   0.9975915  0.9976852 ]]
[[0.99936515 0.99926215 0.9993793  ... 0.99632096 0.99759007 0.9976839 ]]
[[0.99936575 0.99926275 0.99937975 ... 0.9963231  0.9975916  0.9976853 ]]
[[0.9993654  0.9992623  0.9993794  ... 0.99632156 0.99759054 0.99768424]]
[[0.99936527 0.99926215 0.9993793  ... 0.9963212  0.9975903  0.997684  ]]
[[0.99936396 0.9992607  0.9993781  ... 0.9963158  0.9975865  0.9976803 ]]
[[0.99936455 0.9992613  0.99937856 ... 0.996318   0.99758804 0.99768186]]
[[0.9993648  0.9992617  0.9993789  ... 0.99631953 0.9975891  0.9976828 ]]
[[0.99936503 0.9992619  0.99937904 ... 0.99632025 0.9975896  0.9976834 ]]
[[0.99936587 0.99926287 0.9993799  ... 0.9963238  0.9975922  0.9976858 ]]
[[0.9993648  0.99926156 0.9993788  ... 0.99631906 0.9975889  0.9976826 ]]
[[0.9993636  0.99926025 0.9993777  ... 0.996

In [ ]:
temp = np.random.random((1, 24960))
print(temp)
for layer in model.layers:
    weights = layer.get_weights()
    for weight in weights:
        if weight.shape == (512,):
            continue
        print(temp.shape, weight.shape)
        temp = np.dot(temp, weight)
        print(temp)
    print('done')

In [ ]:
temp = model.predict(np.random.random((1, 24960)))
print(temp)

In [ ]:
model.predict(temp)

In [ ]:
model.predict(temp)

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
28*28

In [ ]:
import numpy as np
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))


In [ ]:
x_train.shape

In [ ]:
argmax_y = K.max(K.reshape(logits_y, (-1, N, M)), axis=-1, keepdims=True)
argmax_y = K.equal(K.reshape(logits_y, (-1, N, M)), argmax_y)
encoder = K.function([x], [argmax_y, x_hat])

In [ ]:
code, x_hat_test = encoder([X_test[:100]])
plt.imshow(X_test[1].reshape(120, 208), cmap='gray')
plt.show()
plt.imshow(code[1].reshape(N, M), cmap='gray')
plt.show()
plt.imshow(x_hat_test[1].reshape(120, 208), cmap='gray')
plt.show()

In [ ]:
x_hat_test

In [ ]:
data.shape